In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt

### Базовое знакомство с данными

In [2]:
df = pd.read_csv('/kaggle/input/train-data/train.csv')
df.head()


,id,name,tare
0,0,Котлеты МЛМ из говядины 335г,коробка
1,1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка
2,2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик
3,3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер
4,4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы


In [3]:
df.shape

(40648, 3)

Какие уникальные значения принимает таргет, есть ли дизбаланс?

In [4]:
df["tare"].value_counts()

tare
пакет без формы                   9028
бутылка                           7474
коробка                           4196
пакет прямоугольный               3501
обертка                           3217
банка неметаллическая             2238
стаканчик                         2070
банка металлическая               1837
вакуумная упаковка                1071
усадочная упаковка                 993
контейнер                          884
пачка                              691
лоток                              628
туба                               589
гофрокороб                         419
колбасная оболочка                 396
тортница                           324
без упаковки                       322
упаковка с газовым наполнением     289
ведро                              253
ячеистая упаковка                  228
Name: count, dtype: int64

Разделим выборку на обучающую и тестовую

In [5]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(
    df,
    test_size=0.25,
    stratify=df["tare"]
)

Используем стратифицированное разделение в силу дизбаланса классов.

Делаем мы этого для того, чтобы избежать случайного *выпадания* какого-либо из видов тары. Например, если на трейне к нам не попадут товары, запакованные в тубу, то и на тесте мы их не сможем верно классифицировать. От этого будет страдать обобщающая способность модели. Стратификация позволяет сделать распределение таргетов в трейне и тесте таким, каким оно примерно является в общей совокупности.

Убедимся, что действительно и в трейне, и в тесте схожая доля каждой тары!

In [6]:
train_shares = df_train["tare"].value_counts() / df_train.shape[0]
test_shares = df_test["tare"].value_counts() / df_test.shape[0]

to_compare = pd.concat((train_shares, test_shares), axis=1)
to_compare.columns = ['Доля в трейне', 'Доля в тесте']
to_compare['Абсолютная разница'] = (to_compare["Доля в трейне"] - \
                                    to_compare["Доля в тесте"]).abs()

to_compare

,Доля в трейне,Доля в тесте,Абсолютная разница
tare,,,
пакет без формы,0.222102,0.222102,0.000000
бутылка,0.183855,0.183920,0.000066
коробка,0.103228,0.103228,0.000000
пакет прямоугольный,0.086138,0.086105,0.000033
обертка,0.079151,0.079118,0.000033
банка неметаллическая,0.055042,0.055107,0.000066
стаканчик,0.050941,0.050876,0.000066
банка металлическая,0.045201,0.045168,0.000033
вакуумная упаковка,0.026340,0.026373,0.000033


### Построим базовую модель в качестве бейзлайна. TF-IDF + KNN

Преобразуем наименования товаров с помощью `tf-idf`, взглянем на результат и ровно на нем обучим простейший `KNN`.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

Импортируем классический `TfidfVectorizer` из `sklearn` и обозначим класс за переменную `tfidf`

In [8]:
tfidf = TfidfVectorizer()

Произведем `TfIdf` преобразование на первых 5 наименованиях.

Метод `fit_transform` возвращает `sparse matrix`.

Применим метод `toarray`, чтобы получить данные типа `array`.

In [18]:
tfidf_data = (
    tfidf
    .fit_transform(df["name"].head(10))
    .toarray()
)

tfidf_data

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.4472136 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.4472136 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.4472136 , 0.        , 0.4472136 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.4472136 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.30151134, 0.        , 0.30151134, 0.        ,
        0.     

Отправим полученный `array` в `DataFrame`, чтобы убедиться в корректности работы метода

In [19]:
tfidf_data_df = pd.DataFrame(
    tfidf_data,
    index=df["name"].head(10).index,
    columns=tfidf.get_feature_names_out()
)

tfidf_data_df.head()

,10,100г,12,130гр,140г,190г,20,200г,250г,2л,...,сл,сливочный,сыр,сырный,творог,феличи,чай,чаю,черный,юнимилк
0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.301511,0.0,0.301511,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.447214,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.447214,0.0,0.0,0.0,0.0,0.447214
3,0.0,0.000000,0.0,0.000000,0.0,0.385682,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.327865,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.408248,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


Pipeline сам умеет применять `fit_transform`, поэтому можно так компактно записать процесс `tf-idf` преобразования и обучения на нем модели.

Нет необходимости переводить `array` в `DataFrame`, так как модели из `sklearn` умеют отлично работать с np массивами.


In [20]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

Построим `Pipeline`.

In [21]:
pipeline_baseline = Pipeline(
    [
        ('tfidf_vectorizer', TfidfVectorizer()),
        ('default_KNN', KNeighborsClassifier())
    ]
)

Зафитим модель тренировочными данными и замерим качество на трейне и тесте.

In [26]:
pipeline_baseline.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_baseline.predict(df_train["name"]) 
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_baseline.predict(df_test["name"]) 
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.898
Accuracy на тестовой выборке составило 0.839


Accuracy даже с учетом дисбаланса классов (максимальная доля около 22%) оказывается достаточно высоким.

Есть смысль повалидироваться на гиперпараметрах модели, так как она может в итоге оказаться финально лучшей.

В качестве параметров для валидации выберем:

- Количество соседей (`n`)
- Способ взвешивания соседей (`weights`)
- Параметр p метрики Минковского (`p`)

In [ ]:
from sklearn.model_selection import GridSearchCV

def gaussian_kernel(distances, h=1):
        return np.exp(- distances**2 / h**2)

parameters_grid = {
    'default_KNN__n_neighbors': [5, 10, 20],
    'default_KNN__weights': ['uniform', 'distance', gaussian_kernel],
    'default_KNN__p': (2, 1),
}

search_baseline = GridSearchCV(
    pipeline_baseline,
    parameters_grid,
    scoring="accuracy",
    cv=5,
    verbose=10,
    return_train_score=True
)

search_baseline.fit(df["name"], df["tare"])

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform
[CV 1/5; 1/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform;, score=(train=0.901, test=0.835) total time=   9.0s
[CV 2/5; 1/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform
[CV 2/5; 1/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform;, score=(train=0.900, test=0.844) total time=   9.4s
[CV 3/5; 1/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform
[CV 3/5; 1/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform;, score=(train=0.902, test=0.840) total time=   8.9s
[CV 4/5; 1/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform
[CV 4/5; 1/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform;, score=(train=0.902

Взглянем на лучшую модель и ее качество.

In [28]:
print(f"Best parameter (CV score={search_baseline.best_score_:.5f}):")
print(search_baseline.best_params_)

Best parameter (CV score=0.85810):
{'default_KNN__n_neighbors': 5, 'default_KNN__p': 2, 'default_KNN__weights': <function gaussian_kernel at 0x7921f415d900>}


In [29]:
pipeline_baseline.set_params(**search_baseline.best_params_)

Pipeline(steps=[('tfidf_vectorizer', TfidfVectorizer()),
                ('default_KNN',
                 KNeighborsClassifier(weights=<function gaussian_kernel at 0x7921f415d900>))])

Дополнительно проверим качество лучшей модели (помимо логов `gridsearch`)

In [30]:
pipeline_baseline.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_baseline.predict(df_train["name"]) 
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_baseline.predict(df_test["name"]) 
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.974
Accuracy на тестовой выборке составило 0.858


Качество на трейне выросло, стало почти идеальным +0.076

Качество на тесте тоже выросло, хоть и не так сильно +0.019

Очевидно в глаза бросается переобучение, поэтому есть смысл протестировать модели, менее склонные к нему.

### SVM, RandomForest

Построим пайплайны с тремя предложенными к рассмотрению моделями

In [31]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

pipeline_svm = Pipeline(
    [
        ('tfidf_vectorizer', TfidfVectorizer()),
        ('SVC', SVC())
    ]
)

pipeline_rf = Pipeline(
    [
        ('tfidf_vectorizer', TfidfVectorizer()),
        ('RF', RandomForestClassifier())
    ]
)

Найдем лучшие гиперпараметры для `SVM` и оценим качество на трейне/тесте

In [32]:
svm_parameters_grid = {
    'SVC__C': [1, 0.5, 3],
    'SVC__kernel': ['linear', 'rbf', 'sigmoid']
}

search_svm = GridSearchCV(
    pipeline_svm,
    svm_parameters_grid,
    scoring="accuracy",
    cv=custom_cv,
    return_train_score=True
)

search_svm.fit(df["name"], df["tare"])

pipeline_svm.set_params(**search_svm.best_params_)

pipeline_svm.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_svm.predict(df_train["name"]) 
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_svm.predict(df_test["name"]) 
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.997
Accuracy на тестовой выборке составило 0.875


Качество на тесте выросло +0.017 по сравнению с лучшим KNN

Возможно, стоило лучше поиграться с гиперпараметрыми, например, с `penalty`.

Найдем лучшие гиперпараметры для `RandomForest` и оценим качество на трейне/тесте

In [33]:
rf_parameters_grid = {
    'RF__n_estimators': [10, 100, 200],
    'RF__max_depth': [5, 15, 30, None]
}

search_rf = GridSearchCV(
    pipeline_rf,
    rf_parameters_grid,
    scoring="accuracy",
    cv=custom_cv,
    return_train_score=True
)

search_rf.fit(df["name"], df["tare"])

pipeline_rf.set_params(**search_rf.best_params_)

pipeline_rf.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_rf.predict(df_train["name"]) 
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_rf.predict(df_test["name"]) 
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.998
Accuracy на тестовой выборке составило 0.832


Случайный лес справляется хуже.

### В какую сторону можно искать улучшения?

Во-первых, необходимо лучше обработать текст перед тем, как скармливать его `tf-idf`. Например, такие сущности как 400гр и 0.4кг можно преобразовать к единому формату, то же касается мер объема. Также некоторые названия могут писаться слитно, например, ?*КолбасаДокторская*. В таком случае `tf-idf` распознает это как отдельное уникальное слово, скорее непохожее на просто Колбасу.

Во-вторых, можно продолжить эксперименты с моделями и посмотреть побольше в сторону ансамблей и метрических алгоритмов поверх tf-idf. Или сильнее и глубже поиграться с параметрами регуляризации того же SVM.

Наконец, есть множество других способов классификации текстов: нейросетевой подход, LDA, etc.

## Нейросетевой подход 

In [37]:
!pip install transformers


In [38]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [39]:
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
from datasets import Dataset


# Преобразование меток в числовой формат
label_encoder = LabelEncoder()
df_train["label"] = label_encoder.fit_transform(df_train["tare"])
df_test["label"] = label_encoder.transform(df_test["tare"])

print("Классы:", label_encoder.classes_)

Классы: ['банка металлическая' 'банка неметаллическая' 'без упаковки' 'бутылка'
 'вакуумная упаковка' 'ведро' 'гофрокороб' 'колбасная оболочка'
 'контейнер' 'коробка' 'лоток' 'обертка' 'пакет без формы'
 'пакет прямоугольный' 'пачка' 'стаканчик' 'тортница' 'туба'
 'упаковка с газовым наполнением' 'усадочная упаковка' 'ячеистая упаковка']


In [42]:
# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("chrommium/bert-base-multilingual-cased-finetuned-news-headlines")
model = AutoModelForSequenceClassification.from_pretrained(
    "chrommium/bert-base-multilingual-cased-finetuned-news-headlines",
    num_labels=len(label_encoder.classes_),
    ignore_mismatched_sizes=True
)

# Функция токенизации
def tokenize_function(examples):
    return tokenizer(
        examples["name"],  # Предполагается, что столбец с текстами называется "name"
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Преобразование в Dataset
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# Токенизация
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Удаление ненужных столбцов
tokenized_train_dataset = tokenized_train_dataset.remove_columns([col for col in ["name", "tare"] if col in tokenized_train_dataset.column_names])
tokenized_test_dataset = tokenized_test_dataset.remove_columns([col for col in ["name", "tare"] if col in tokenized_test_dataset.column_names])

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at chrommium/bert-base-multilingual-cased-finetuned-news-headlines and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([21, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([21]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/30486 [00:00<?, ? examples/s]

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

In [44]:
# Настройки обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

# Создание Trainer
# Загрузка метрики accuracy
metric = evaluate.load("accuracy")

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Обучение модели
trainer.train()

# Оценка модели на тестовой выборке
eval_results = trainer.evaluate(tokenized_test_dataset)

# Вывод accuracy на тестовой выборке
accuracy = eval_results['eval_accuracy']
print(f"Accuracy на тестовой выборке: {accuracy}")

# Вывод результатов
print("Результаты оценки:", eval_results)

Epoch,Training Loss,Validation Loss,Accuracy
1,1.168400,1.107966,0.684413
2,0.825800,0.850604,0.753001
3,0.698400,0.756112,0.786656
4,0.576400,0.705894,0.807518
5,0.298600,0.716229,0.812832


Accuracy на тестовой выборке: 0.8075182050777406
Результаты оценки: {'eval_loss': 0.705893874168396, 'eval_accuracy': 0.8075182050777406, 'eval_runtime': 70.7899, 'eval_samples_per_second': 143.552, 'eval_steps_per_second': 8.984, 'epoch': 5.0}
